# Data Preparation (from .xml to .tfrecord)

In [1]:
import os, glob, pandas as pd, xml.etree.ElementTree as ET, tensorflow as tf, cv2, numpy as np

from collections import namedtuple

In [2]:
def read_label_map(label_map_path):
    item_id = None
    item_name = None
    items = {}

    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").replace("\"", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

In [3]:
def class_text_to_num(label_map_dict, text):
    return label_map_dict[text]

In [4]:
def regroup(dataframe, group = "filename"):
    data = namedtuple("data", ["filename", "objects"])
    group_by = dataframe.groupby(group)
    return [data(filename, group_by.get_group(x)) for filename, x in zip(group_by.groups.keys(), group_by.groups)]

In [5]:
def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [6]:
def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

## Dataset

In [7]:
label_map = "./label_map.pbtxt"
label_map_dict = read_label_map(label_map)

label_map_dict

{'pothole': 1, 'crack': 2}

In [8]:
def xml_to_csv(path, label_map):
    xml_list = []
    for file in glob.glob(path + "/*.xml"):
        tree = ET.parse(file)
        root = tree.getroot()
        members = root.findall("object")
        objects_class = [member[0].text for member in members]
        is_pothole = "pothole" in objects_class
#         print(root.find("filename").text)
        for member in members:
            if member[0].text == "plain": continue
            if is_pothole:
                if member[0].text == "crack": continue
            xml_list.append((
                root.find("filename").text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
                int(class_text_to_num(label_map, member[0].text)),
            ))  # filename, x1, y1, x2, y2, class
#             else: continue
    columns = ["filename", "x1", "y1", "x2", "y2", "class"]
    return pd.DataFrame(xml_list, columns = columns)

In [9]:
def create_tf_ex(group, path):
    # Color
    image = cv2.imread(os.path.join(path, group.filename), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # end of Color
    
    # Grayscale
#     image = cv2.imread(os.path.join(path, group.filename), cv2.IMREAD_GRAYSCALE)
#     image = image.reshape(image.shape[0], image.shape[1], 1)
    # end of Grayscale
    
    filename = group.filename.encode("utf-8")
    label = []
    bbox = []
    for i, data in group.objects.iterrows():
        bbox.append([float(data["y1"] / 512), float(data["x1"] / 512), float(data["y2"] / 512), float(data["x2"] / 512)])
        label.append(data["class"])
    bbox = np.array(bbox)
    bbox = bbox.astype("float32")
    features = tf.train.Features(feature = {
        "filename": bytes_feature(filename),
        "pic": bytes_feature(tf.io.serialize_tensor(image)),
        "bbox": bytes_feature(tf.io.serialize_tensor(bbox)),
        "label": bytes_feature(tf.io.serialize_tensor(np.array(label))),
    })
    return tf.train.Example(features = features)

In [10]:
def create_tf_record(group_x, path, writer):
    i = 0
    for group in group_x:
        tf_ex = create_tf_ex(group, path)
        writer.write(tf_ex.SerializeToString())
        i += 1
    writer.close()
    print("DONE {}\n{}".format(path, i))

In [11]:
path_train = "../dataset/train/augmented"
path_test = "../dataset/test/augmented"

train_out = "./train.tfrecord"
test_out = "./test.tfrecord"

writer_train = tf.io.TFRecordWriter(train_out)
writer_test = tf.io.TFRecordWriter(test_out)

df_train = xml_to_csv(path_train, label_map_dict)
df_test = xml_to_csv(path_test, label_map_dict)

group_train = regroup(df_train)
group_test = regroup(df_test)

In [12]:
create_tf_record(group_train, path_train, writer_train)
create_tf_record(group_test, path_test, writer_test)

DONE ../dataset/train/augmented
1950
DONE ../dataset/test/augmented
480


## Dataset ROI

In [ ]:
def create_roi_tf_ex(group, path, label_map):
    # Color
    image = cv2.imread(os.path.join(path, group.filename), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # end of Color

    # Grayscale
    # image = cv2.imread(os.path.join(path, group.filename), cv2.IMREAD_GRAYSCALE)
    # image = image.reshape(image.shape[0], image.shape[1], 1)
    # end of Grayscale

    # image = open(os.path.join(path, group.filename), "rb").read()

    label = -1
    filename = group.filename.encode("utf-8")
    for i, data in group.objects.iterrows():
        label = class_text_to_num(label_map, data["class"]) - 1

    features = tf.train.Features(feature = {
        "filename": bytes_feature(filename),
        "image": bytes_feature(tf.io.serialize_tensor(image)),
        "label": int64_feature(label),
    })
    return tf.train.Example(features = features).SerializeToString()

In [ ]:
def create_tf_record_roi(group_x, path, label_map_roi, writer):
    total = 0
    for group in group_x:
        writer.write(create_roi_tf_ex(group, path, label_map_roi))
        total += 1
    writer.close()
    print("DONE {}\n{}".format(path, total))

In [ ]:
label_map_roi_path = "./label_map.pbtxt"
label_map_roi = read_label_map(label_map_roi_path)
print(label_map_roi)

In [ ]:
train_out_roi = "./train_roi.tfrecord"
test_out_roi = "./test_roi.tfrecord"

train_writer = tf.io.TFRecordWriter(train_out_roi)
test_writer = tf.io.TFRecordWriter(test_out_roi)

train_roi_path = "../dataset/train_roi/ready"
test_roi_path = "../dataset/test_roi/ready"

train_df = pd.read_csv(train_roi_path + "/train.csv", encoding='utf-8')
test_df = pd.read_csv(test_roi_path + "/test.csv", encoding='utf-8')

train_group = regroup(train_df)
test_group = regroup(test_df)

In [ ]:
create_tf_record_roi(train_group, train_roi_path, label_map_roi, train_writer)
create_tf_record_roi(test_group, test_roi_path, label_map_roi, test_writer)